# Quickstart

This notebook contains the code examples from the [readme](../README.md).

In [1]:
import enact
import dataclasses

# Allow re-registration of resources to avoid errors when re-running cells.
enact.Registry.get().allow_reregistration = True

/home/leo/dev/enact/.venv/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Defining and storing custom resources

In [2]:
@enact.register
@dataclasses.dataclass
class MyResource(enact.Resource):
  x: int
  y: float

with enact.Store() as store:
  ref = enact.commit(MyResource(42, 69.0))
  print(ref.id)
  print(ref.get())

{"digest": "11c8fb43c28c1894a6b14841ae985c526ac907086784aa43a9fab94b6ee08443"}
MyResource(x=42, y=69.0)


## Invoking resources

In [3]:
@enact.typed_invokable(input_type=enact.NoneResource, output_type=MyResource)
class MyInvokable(enact.Invokable):

  def call(self):
    return MyResource(42, 69.0)

with store:
  my_invokable = MyInvokable()
  # Simple execution:
  print(my_invokable())  # Prints "MyResource(x=42, y=69.0)".
  # Tracked execution:
  invocation = my_invokable.invoke()
  enact.pprint(invocation)

MyResource(x=42, y=69.0)
Invocation:
  request:
    -> Request#fb29d4:
      invokable: -> MyInvokable()#3e4b9b
      input: -> NoneResource()#4c96e8
  response:
    -> Response#32ae3d:
      invokable: -> MyInvokable()#3e4b9b
      output: -> MyResource(x=42, y=69.0)#11c8fb
      raised: None
      raised_here: False
      children: []


## Creating UIs

In [4]:
with store:
  ref = enact.commit(my_invokable)
  enact.GUI(ref).launch(share=True)

Running on local URL:  http://127.0.0.1:7860
Running on public URL: https://4c53526d1ce6c3aabc.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


### Requesting inputs and replaying invocations

In [5]:
@enact.typed_invokable(input_type=enact.NoneResource, output_type=MyResource)
class SampleFromHuman(enact.Invokable):

  def call(self):
    request_int = enact.RequestInput(enact.Int)
    request_float = enact.RequestInput(enact.Float)
    return MyResource(
      x=request_int(enact.Str('Please provide an x-value for MyResource.')),
      y=request_float(enact.Str('Please provide a y-value for MyResource.')))

with store:
  h = SampleFromHuman()
  # Run until first input request.
  invocation = h.invoke()
  # Access InputRequest exception.
  input_request = invocation.response().raised()
  print(input_request.input())  # Prints 'Please provide an x-value ...'.
  # Run until second input request.
  invocation = input_request.continue_invocation(invocation, enact.Int(42))
  # Access InputRequest exception.
  input_request = invocation.response().raised()
  print(input_request.input())  # Prints 'Please provide a y-value ...'.
  # Run until completion.
  invocation = input_request.continue_invocation(invocation, enact.Float(69.0))
  print(invocation.response().output())  # Prints 'MyResource(x=42, y=69.0)'.

Please provide an x-value for MyResource.
Please provide a y-value for MyResource.
MyResource(x=42, y=69.0)


The above is roughly equivalent to:

In [6]:
with store:
  # Run until first exception.
  invocation = h.invoke()
  def override_exception(exc_ref):
    if exc_ref().requested_type == enact.Int:
      return enact.Int(42)
    if exc_ref().requested_type == enact.Float:
      return enact.Float(69.0)
  # Inject first value and run until second exception.
  invocation = invocation.replay(override_exception)
  # Inject second value and run until completion.
  invocation = invocation.replay(override_exception)
  print(invocation.response().output())  # Prints 'MyResource(x=42, y=69.0)'.

MyResource(x=42, y=69.0)


UIs can handle input requests automatically. The required types needs to be
specified on launch.

In [7]:
with store:
  ref = enact.commit(h)
  enact.GUI(ref, input_required_inputs=[enact.Int, enact.Float]).launch(
    share=True)

Running on local URL:  http://127.0.0.1:7861
Running on public URL: https://d2ae07f2034f4107e7.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)
